# Importing libraries, Examining data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Uploading csv

df = pd.read_csv('https://raw.githubusercontent.com/Bomsk/DA3_2023/main/Assignment%201/morg-2014-emp.csv?token=GHSAT0AAAAAACMSJ4X4SZCXPWSOLSNWGV36ZNDJ5IA')

/var/folders/3g/n4f0cyks6h5d928zs76xn4x40000gn/T/ipykernel_18304/3169933115.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://raw.githubusercontent.com/Bomsk/DA3_2023/main/Assignment%201/morg-2014-emp.csv?token=GHSAT0AAAAAACMSJ4X4SZCXPWSOLSNWGV36ZNDJ5IA')


In [3]:
df.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work


In [ ]:
# Filtering for Computer and mathematical science occupations
# code: 1000-1240

